In [1]:
import numpy as np
import open3d as o3d
import laspy
from typing import Union, Tuple

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# 读取 PLY 文件
col_pcd = o3d.io.read_point_cloud("../3d_model/colmap_glacier.ply")
# col_pcd = o3d.io.read_point_cloud("../3d_model/filtered_point_cloud.ply")
len(col_pcd.points)

520723

In [2]:
las_file = laspy.read("../3d_model/2016-11-28_AlphLake_SFM-Medium_USGSTransLOCAL.las")

# 提取 XYZ 坐标
las_points = np.vstack((las_file.x, las_file.y, las_file.z)).T

# 转换为 Open3D 点云对象
las_pcd = o3d.geometry.PointCloud()
las_pcd.points = o3d.utility.Vector3dVector(las_points)
len(las_pcd.points)

13178690

In [3]:
o3d.io.write_point_cloud("../3d_model/gt_gl.ply", las_pcd)

True

In [4]:
gl_pcd = o3d.io.read_point_cloud("../3d_model/gt_gl.ply")
len(gl_pcd.points)

13178690

In [4]:
def get_bounding_box_range(points):
    min_vals = np.min(points, axis=0)
    max_vals = np.max(points, axis=0)
    return max_vals - min_vals

In [5]:
range1 = get_bounding_box_range(col_pcd.points)
range2 = get_bounding_box_range(las_pcd.points)

In [6]:
print("colmap range: ", range1)
print("las range: ", range2)

colmap range:  [359.34918213 325.8598175   66.49077129]
las range:  [65.5267 57.1422 17.1178]


In [7]:
print("Apply point-to-point ICP")
threshold = 1000
trans_init = np.asarray([[0.862, 0.011, -0.507, 0.5],
                         [-0.139, 0.967, -0.215, 0.7],
                         [0.487, 0.255, 0.835, -1.4], 
                         [0.0, 0.0, 0.0, 1.0]])
reg_p2p = o3d.pipelines.registration.registration_icp(
    col_pcd, las_pcd, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint())
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)

Apply point-to-point ICP
RegistrationResult with fitness=1.000000e+00, inlier_rmse=4.350983e+00, and correspondence_set size of 520723
Access transformation to get result.
Transformation is:
[[ 7.52874643e-01 -5.75169302e-01 -3.20793725e-01  1.43258266e+02]
 [ 6.03092501e-01  7.97114878e-01 -1.29597075e-02 -6.99851886e+01]
 [ 2.62665581e-01 -1.84453639e-01  9.47323826e-01  1.82368236e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [8]:
col_pcd.transform(reg_p2p.transformation)

PointCloud with 520723 points.

In [9]:
from pytorch3d.loss import chamfer_distance
import torch

In [10]:
# 提取点云坐标，并转换为 NumPy 数组
source_points = np.asarray(col_pcd.points)
target_points = np.asarray(las_pcd.points)

# 转换 NumPy 数组为 PyTorch Tensor
source_tensor = torch.tensor(source_points, dtype=torch.float32).unsqueeze(0)  # (1, N, 3)
target_tensor = torch.tensor(target_points, dtype=torch.float32).unsqueeze(0)  #

In [ ]:
# By default: average over points then batches
loss_chamfer, (dist_src_tgt, dist_tgt_src) = chamfer_distance(
    source_tensor, target_tensor,
    batch_reduction="mean",    # or "sum" / "none"
    point_reduction="mean"     # or "sum" / "none"
)

print(f"Chamfer loss: {loss_chamfer.item():.6f}")
# dist_src_tgt: tensor of shape (B, N) – for each src point its squared L2 to nearest tgt
# dist_tgt_src: tensor of shape (B, M) – for each tgt point its squared L2 to nearest src
